In [76]:
from model2 import *
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [96]:
class DateTransformer():
    """Transforms df so that there are columns representing the features of the date.
    """
    
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X = pd.Series([pd.to_datetime(x, format="%Y-%m-%d") for x in X['date']])
        X_transf = pd.DataFrame()
        X_transf['year'] = [int(x.year) for x in X]
        X_transf['month'] = [int(x.month) for x in X]
        X_transf['day'] = [int(x.day) for x in X]
        X_transf['weekday'] = [int(x.weekday()) for x in X]
        return np.array(X_transf)

In [97]:
df = pd.read_sql_query("""
                        SELECT latitude, 
                                longitude, 
                                ap_t_high100, 
                                n_arrests,
                                date
                        FROM manhattan_loc_d_ar_wea 
                        ;"""
                        , 'postgresql:///walk')

In [98]:
categorized_df = categorize_arrests(df)

In [99]:
    # add combined lat/long location feature
    categorized_df['latlong'] = (categorized_df['latitude'].astype(str) 
                                + categorized_df['longitude'].astype(str))
    X_train, y_train, X_eval, y_eval = split_last(categorized_df, target_col='n_arrests',
                                                  sort_col=None, cut=.8)

    

In [100]:
X_train.head()

,latitude,longitude,ap_t_high100,date,latlong
0,40.683,-74.023,1943,2018-01-02,40.683-74.023
1,40.684,-74.012,1943,2018-01-02,40.684-74.012
2,40.684,-74.024,1943,2018-01-02,40.684-74.024
3,40.685,-74.022,1943,2018-01-02,40.685-74.022
4,40.685,-74.023,1943,2018-01-02,40.685-74.023


In [ ]:
date_ohe_categories = [
    # year categories
    [2018, 2019],
    # month categories
    list(range(0, 31, 1))
    # day categories
    []
    # weekday categories
    []
]

In [114]:
date_transformer = Pipeline(steps=[
    ('date_expansion', DateTransformer()),
    ('ohe', OneHotEncoder(categories=date_ohe_categories))
])

NameError: name 'date_ohe_categories' is not defined

In [102]:
    column_transformer = ColumnTransformer( 
        transformers=[
        ('ohe', OneHotEncoder(categories='auto'), ['latlong']),
        ('ss', StandardScaler(), ['latitude', 'longitude', 'ap_t_high100']),
        ('date', date_transformer, ['date']),
        ], remainder='drop')

    

In [103]:
        stamp_time()
        rfc = RandomForestClassifier(n_estimators=10, max_depth=10,
                                class_weight='balanced')

Current Time = 10:21:03


In [113]:
        pipe = Pipeline([
            ('preprocessor', column_transformer),
            ('model', rfc)
        ])

        log_loss_cvs(pipe, X_train, y_train)
        stamp_time()

ValueError: Found unknown categories [4, 5] in column 1 during transform

In [106]:
X_train_transformed = column_transformer.fit_transform(X_train)

In [107]:
rfc.fit(X_train_transformed, y_train)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
                       criterion='gini', max_depth=10, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=10, n_jobs=None, oob_score=False,
                       random_state=None, verbose=0, warm_start=False)

In [109]:
X_eval_transformed = column_transformer.transform(X_eval)

In [110]:
    train_probas = rfc.predict_proba(X_train_transformed)
    test_probas = rfc.predict_proba(X_eval_transformed)
    test_predict = rfc.predict(X_eval_transformed)

In [111]:
    print(f"training log loss: {log_loss(y_train, train_probas)}") 
    print(f"test neg log loss: {log_loss(y_eval, test_probas)}")
    print(f"confusion matrix: \n{confusion_matrix(y_eval, test_predict)}")
    print(f"accuracy: {accuracy_score(y_eval, test_predict)}")
    print(f"F1: {f1_score(y_eval, test_predict)}")

training log loss: 0.673805066741027
test neg log loss: 0.6733167630821818
confusion matrix: 
[[503444 173407]
 [  5777   5380]]
accuracy: 0.7395611678933965
F1: 0.05664827528113549


In [112]:
pickle_model(rfc)